In [1]:
import pandas as pd
from datetime import datetime
import requests
import time 
import json
from requests_html import HTMLSession
from PIL import Image
from io import StringIO, BytesIO

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from ipywidgets import IntProgress
from IPython.display import display, clear_output

%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [3]:
! pip install --upgrade selenium

In [16]:
!pip install jupyternotify

In [100]:
!pip install tqdm

In [2]:
session = HTMLSession()

In [38]:
%%notify
start = time.time()
pages = 100
single_page_urls = []
page_cnt, url_cnt = 0, 0
category = 'electronics'
for p in range(1, pages+1):
    bikroy_url = f'https://bikroy.com/en/ads/bangladesh/{category}?sort=date&order=desc&buy_now=0&urgent=0&page={p}'
    print(bikroy_url)
    r = session.get('http://localhost:8055/render.html', params={'url': bikroy_url, 'wait': 1})

    page_cnt += 1
    for i in range (1, 28):
        try:
#             single_url = driver.find_element(by=By.XPATH, value=f'//*[@id="app-wrapper"]/div[1]/div[3]/div/div[2]/div[4]/div[2]/div/div[1]/div[1]/ul/li[{i}]/a').get_attribute('href')
            single_url = r.html.find(f'#app-wrapper > div.container--297Nx.all > div:nth-child(3) > div > div.row--3Vhjr.main-container--28aG5.justify-content-flex-start--1Xozy.align-items-normal--vaTgD.flex-wrap-wrap--2PCx8.flex-direction-row--27fh1.flex--3fKk1 > div.sm-col-12--30zDS.lg-col-9--20qCf.block--3v-Ow > div.ad-list-container--1UnyA > div > div.ad-list--2Y3ql > div.list-wrapper--t_A02 > ul > li:nth-child({i}) > a')
            single_url = single_url[0].attrs['href']
            single_page_urls.append(single_url)
            url_cnt += 1
            print("Page: " + str(page_cnt) + " URL: " + str(url_cnt))
        except:
            print('Not details link')
    
    clear_output(wait=True)
print(time.time()-start)
# driver.maximize_window()

241.41733026504517


<IPython.core.display.Javascript object>

In [40]:
len(single_page_urls)

2700

In [39]:
with open(f'category_url/{category}.txt', 'w') as f:
    for s in single_page_urls:
        f.write(s)
        f.write('\n')

In [2]:
def features_collection(text, easy=False):
    data_dic = {}
    if easy:
        ad_data = text["ad"]
    else:
        ad_data = text["adDetail"]["data"]["ad"]
    ad_id = ad_data['id']
    title = ad_data["title"]
    description = ad_data["description"]
    properties = ad_data["properties"]
    features = {}
    for p in properties:
        features[p["label"]] = p["value"]
#     print('Properties', properties)
#     print('Features', features)
    is_member = ad_data["isMember"]
    is_auth_dealer = ad_data["isAuthDealer"]
    is_featured_member = ad_data["isFeaturedMember"]
    membership_level = ad_data["membershipLevel"]
    is_verified = ad_data["isVerified"]
    door_step_delivery = ad_data["isDoorstepDelivery"]
    dsd_applicable = ad_data["isDSDApplicable"]
    try:
        timestamp = ad_data["timestamp"]
        date = ad_data["adDate"]
    except:
        timestamp = 'N/A'
        date = 'N/A'
    deactivates = ad_data["deactivates"]
    location_data = ad_data["location"]
    location = location_data["name"]
    geography_data = location_data["geography"]["simple_polygons"][0]["coordinates"]
    geo_polygon_no = len(geography_data)

    category_name = ad_data["category"]["name"]
    area = ad_data["area"]["name"]
    contact_name = ad_data["contactCard"]["name"]
    contact_number = ad_data["contactCard"]["phoneNumbers"]
    phone_number = [p["number"] for p in contact_number]
    
    try:
        shop_name = ad_data["shop"]["name"]
    except:
        shop_name = 'N/A'
    try:
        shop_email = ad_data["shop"]["email"]
    except:
        shop_email = 'N/A'

    price = ad_data["money"]["amount"]
    delivery_free = ad_data["isDeliveryFree"]

    if easy:
        similar_ads = text["similarAds"]
    else:
        similar_ads = text["adDetail"]["data"]["similarAds"]

    image_meat_urls = ad_data['images']['meta']
    image_urls = [i['src']+'/1200/800/fitted.jpg' for i in image_meat_urls]
#     print(image_urls)
    
    img_path = []   
    for img_url in image_urls:
        r = requests.get(img_url)
        i = Image.open(BytesIO(r.content))
        img_name = '@'.join(img_url.split('/')[3:5])
        i.save(f"img/{category}/{img_name}.jpg")
        img_path.append(f"img/{category}/{img_name}.jpg")
    
    data_dic['ad_id'] = ad_id
    data_dic['title'] = title
    data_dic['description'] = description
    data_dic['features'] = features
    data_dic['is_member'] = is_member
    data_dic['is_auth_dealer'] = is_auth_dealer
    data_dic['is_featured_member'] = is_featured_member
    data_dic['membership_level'] = membership_level
    data_dic['is_verified'] = is_verified
    data_dic['door_step_delivery'] = door_step_delivery
    data_dic['dsd_applicable'] = dsd_applicable
    data_dic['timestamp'] = timestamp
    data_dic['date'] = date
    data_dic['deactivates'] = deactivates
    data_dic['geography_data'] = geography_data
    data_dic['geo_polygon_no'] = geo_polygon_no
    data_dic['area'] = area
    data_dic['contact_name'] = contact_name
    data_dic['phone_number'] = phone_number
    data_dic['shop_name'] = shop_name
    data_dic['shop_email'] = shop_email
    data_dic['price'] = price
    data_dic['delivery_free'] = delivery_free
    data_dic['similar_ads'] = similar_ads
    data_dic['image_url'] = image_urls
    data_dic['image_path'] = img_path
    
    return data_dic

In [3]:
%%notify
category = 'mobile'
session = HTMLSession()

with open(f'category_url/{category}.txt', 'r') as f:
    single_url = f.readlines()
    titles, locations, prices, img_lst, meta_lst,  feature_lst, desc_lst = [], [], [], [], [], [], []
    cnt = 1
    all_data = []
    failed_list = []
    easy_data_cnt = 0
    full_data_cnt = 0
    max_count = len(single_url)
    f = IntProgress(min=0, max=max_count) # instantiate the bar
    display(f) # display the bar

    for s in single_url:
        try:
            s = s.replace('\n', '')
            slug = s.split('/')[-1]
            url = 'https://bikroy.com/data/details?slug='+slug+'&similar_ads=16'
            r = session.get('http://localhost:8055/render.html', params={'url': url})
            text = r.html.text 
            if 'Error retrieving ad details' in text:
                r = session.get('http://localhost:8055/render.html', params={'url': s, 'wait': 2})
                print(cnt, s)

                text = r.html.text 

                a_split = text.split('window.initialData = ')[1][:-1]
                b_split = a_split.split('"@context":') #"@context": "http://schema.org",
                data = b_split[0][:-3] + ',' + '"@context": ' + b_split[1][:-2]
                data = data.replace('\\', '')
                jtext = json.JSONDecoder().decode(data)

                # attributes extraction
                data_dic = features_collection(jtext)
                full_data_cnt += 1
            else:
                print(cnt, url)
                text = r.html.text 
                text = json.JSONDecoder().decode(text)
                data_dic = features_collection(text, True)
                easy_data_cnt += 1
                
            data_dic['url'] = s
            all_data.append(data_dic)

            cnt += 1
            print("=============================================")
#             clear_output(wait=True)
            f.value += 1 # signal to increment the progress bar
            
            
            
        except:
            print(cnt, s)
            print('Failed')
            failed_list.append(s)
            print('------------------')

IntProgress(value=0, max=2268)

1 https://bikroy.com/en/ad/oneplus-9-pro-12gb-256gb-global-new-for-sale-chattogram
2 https://bikroy.com/data/details?slug=oppo-a57-3-32-new-for-sale-barishal-7&similar_ads=16
3 https://bikroy.com/data/details?slug=samsung-galaxy-j6-3-32-used-for-sale-dhaka-770&similar_ads=16
4 https://bikroy.com/en/ad/realme-c11-used-for-sale-sylhet-division-12
5 https://bikroy.com/data/details?slug=samsung-galaxy-a7-2018-used-for-sale-dhaka-1198&similar_ads=16
6 https://bikroy.com/data/details?slug=samsung-galaxy-j2-15gb-32gb-used-for-sale-khulna-division&similar_ads=16
7 https://bikroy.com/en/ad/xiaomi-redmi-note-7-pro-4-64-used-for-sale-khulna-327
8 https://bikroy.com/data/details?slug=samsung-galaxy-a20s-4-64-used-for-sale-chattogram-division-14&similar_ads=16
9 https://bikroy.com/en/ad/samsung-galaxy-s9-plus-2018-used-for-sale-dhaka-95
10 https://bikroy.com/data/details?slug=symphony-b6-used-for-sale-dhaka-595&similar_ads=16
11 https://bikroy.com/data/details?slug=samsung-2021-used-for-sale-chatto

56 https://bikroy.com/en/ad/xiaomi-redmi-3261-used-for-sale-mymensingh-division
57 https://bikroy.com/en/ad/oppo-a5s-a7-4-64-used-for-sale-dhaka
58 https://bikroy.com/data/details?slug=xiaomi-redmi-note-6-pro-used-good-used-for-sale-chattogram&similar_ads=16
59 https://bikroy.com/en/ad/xiaomi-redmi-note-10-pro-max-snapdragon-732g-used-for-sale-rajshahi
60 https://bikroy.com/data/details?slug=samsung-galaxy-a20-3-32fresh-used-for-sale-dhaka&similar_ads=16
61 https://bikroy.com/en/ad/xiaomi-mi-a2-used-for-sale-dhaka-2767
62 https://bikroy.com/data/details?slug=oneplus-9r-8-256like-newbox-used-for-sale-dhaka&similar_ads=16
62 https://bikroy.com/en/ad/oneplus-9r-8-256like-newbox-used-for-sale-dhaka
Failed
------------------
62 https://bikroy.com/data/details?slug=itel-vision-1-used-for-sale-dhaka-division-160&similar_ads=16
63 https://bikroy.com/data/details?slug=xiaomi-poco-x3-pro-2021-used-for-sale-khulna-7&similar_ads=16
64 https://bikroy.com/data/details?slug=apple-iphone-7-plus-256gb-

105 https://bikroy.com/data/details?slug=6-128-used-for-sale-barishal&similar_ads=16
106 https://bikroy.com/en/ad/apple-iphone-7-used-for-sale-sylhet-1304
107 https://bikroy.com/data/details?slug=samsung-a124-64official-used-for-sale-dhaka-2&similar_ads=16
108 https://bikroy.com/en/ad/xiaomi-redmi-7-mobile-phone-used-for-sale-dhaka-1
109 https://bikroy.com/data/details?slug=samsung-galaxy-a03s-a03-new-for-sale-dhaka&similar_ads=16
110 https://bikroy.com/en/ad/xiaomi-redmi-note-5-4-64-used-for-sale-dhaka-1047
111 https://bikroy.com/en/ad/vivo-y30-used-for-sale-dhaka-456
112 https://bikroy.com/data/details?slug=realme-x2-c21-used-for-sale-dhaka-4&similar_ads=16
113 https://bikroy.com/data/details?slug=samsung-a10s-2-32-used-for-sale-chattogram-division-96&similar_ads=16
114 https://bikroy.com/data/details?slug=apple-iphone-12-pro-max-6-256-used-for-sale-chattogram-division-1&similar_ads=16
115 https://bikroy.com/data/details?slug=oneplus-nord-n100-4-64full-freshbox-used-for-sale-dhaka&si

159 https://bikroy.com/data/details?slug=samsung-galaxy-grand-prime-plus-used-for-sale-dhaka-2328&similar_ads=16
160 https://bikroy.com/en/ad/apple-iphone-5c-used-for-sale-sylhet-501
161 https://bikroy.com/en/ad/xiaomi-redmi-note-6-pro-nstt-phon-kinte-caai-to-buy-rajshahi
161 https://bikroy.com/en/ad/xiaomi-redmi-note-6-pro-nstt-phon-kinte-caai-to-buy-rajshahi
Failed
------------------
161 https://bikroy.com/data/details?slug=iphone-6plus-front-camera-home-botton-rebon-drkr-to-buy-mymensingh&similar_ads=16
161 https://bikroy.com/en/ad/iphone-6plus-front-camera-home-botton-rebon-drkr-to-buy-mymensingh
Failed
------------------
161 https://bikroy.com/en/ad/mobaail-phon-kryy-bikryy-ebn-ekcenj-to-buy-rangpur-division
161 https://bikroy.com/en/ad/mobaail-phon-kryy-bikryy-ebn-ekcenj-to-buy-rangpur-division
Failed
------------------
161 https://bikroy.com/en/ad/samsung-chat-gt-c3312-used-for-sale-dhaka
162 https://bikroy.com/data/details?slug=vivo-y12s-good-condition-used-for-sale-rajshahi-di

...

1932 https://bikroy.com/en/ad/vivo-y-67432g7-maarc-new-for-sale-rangpur-division
1932 https://bikroy.com/en/ad/vivo-y-67432g7-maarc-new-for-sale-rangpur-division
Failed
------------------
1932 https://bikroy.com/en/ad/xiaomi-redmi-note-11-4-64-new-for-sale-sylhet-1
1933 https://bikroy.com/data/details?slug=samsung-m02s-used-for-sale-dhaka-91&similar_ads=16
1934 https://bikroy.com/en/ad/huawei-p30-lite-6-gb-used-for-sale-dhaka-2
1935 https://bikroy.com/en/ad/mi-band-6-for-sale-dhaka-27
1935 https://bikroy.com/en/ad/mi-band-6-for-sale-dhaka-27
Failed
------------------
1935 https://bikroy.com/en/ad/huawei-honor-6-128gb-used-for-sale-dhaka-1
1936 https://bikroy.com/en/ad/xiaomi-redmi-note-8-1-used-for-sale-khulna-division-4
1937 https://bikroy.com/en/ad/huawei-y5-fresh-condition-used-for-sale-dhaka-1
1938 https://bikroy.com/en/ad/xiaomi-redmi-5-used-for-sale-rangpur-264
1939 https://bikroy.com/data/details?slug=xiaomi-redmi-note-7s-used-for-sale-khulna-71&similar_ads=16
1940 https://bikro

1985 https://bikroy.com/en/ad/samsung-galaxy-a50s-used-for-sale-rajshahi-145
1986 https://bikroy.com/data/details?slug=samsung-galaxy-j2-pro-2-16-used-for-sale-khulna-division-41&similar_ads=16
1987 https://bikroy.com/en/ad/dongol-for-sale-barishal
1988 https://bikroy.com/data/details?slug=symphony-i10-bhaaleaa-used-for-sale-rangpur&similar_ads=16
1989 https://bikroy.com/en/ad/apple-iphone-x-used-for-sale-dhaka-5360
1990 https://bikroy.com/en/ad/samsung-m40-2021-6-128-used-for-sale-rajshahi
1991 https://bikroy.com/data/details?slug=realme-c17-6-128-full-box-used-for-sale-khulna-division-1&similar_ads=16
1992 https://bikroy.com/data/details?slug=oppo-f19-pro-8-128gb-used-for-sale-dhaka-67&similar_ads=16
1993 https://bikroy.com/data/details?slug=xiaomi-mi-11x-5g-6-128-used-for-sale-rajshahi-13&similar_ads=16
1994 https://bikroy.com/en/ad/xiaomi-mi-a3-4-64-used-for-sale-khulna-division-106
1995 https://bikroy.com/en/ad/huawei-y5-2021-1-16-y5-lite-used-for-sale-sylhet
1996 https://bikroy.c

2043 https://bikroy.com/en/ad/xiaomi-redmi-note-10-used-used-for-sale-sylhet
2044 https://bikroy.com/data/details?slug=samsung-m02-3-32gb-used-for-sale-dhaka-13&similar_ads=16
2045 https://bikroy.com/en/ad/sony-xperia-xz-2-compact-4-64-used-for-sale-dhaka-3
2046 https://bikroy.com/data/details?slug=realme-x2-8-128-used-for-sale-dhaka-383&similar_ads=16
2047 https://bikroy.com/data/details?slug=xiaomi-poco-x3-8gb-128gb-used-for-sale-dhaka-27&similar_ads=16
2048 https://bikroy.com/data/details?slug=xiaomi-redmi-note-7-pro-4-gb-64-used-for-sale-rajshahi-division-1&similar_ads=16
2049 https://bikroy.com/data/details?slug=xiaomi-redmi-note-9-6-128speaker-pblm-used-for-sale-dhaka&similar_ads=16
2050 https://bikroy.com/en/ad/xiaomi-redmi-note-7-pro-use-mobile-used-for-sale-khulna-2
2051 https://bikroy.com/data/details?slug=oppo-f5-4gb-6gb-new-for-sale-dhaka&similar_ads=16
2052 https://bikroy.com/data/details?slug=vivo-y21-used-for-sale-rangpur-division-13&similar_ads=16
2053 https://bikroy.co

2100 https://bikroy.com/en/ad/xiaomi-redmi-9-2021-used-for-sale-dhaka-60
2101 https://bikroy.com/en/ad/vivo-y67a-4gb-32gb-today-offer-new-for-sale-dhaka-1
2102 https://bikroy.com/data/details?slug=nokia-106-used-for-sale-dhaka-620&similar_ads=16
2103 https://bikroy.com/en/ad/oneplus-7-pro-5g-used-for-sale-mymensingh-9
2104 https://bikroy.com/en/ad/apple-iphone-13-pro-max-256-gb-new-for-sale-chattogram-5
2105 https://bikroy.com/data/details?slug=xiaomi-redmi-5-7th-march-offer-new-for-sale-dhaka&similar_ads=16
2106 https://bikroy.com/data/details?slug=xiaomi-redmi-note-9-4-128-used-for-sale-rajshahi-division-50&similar_ads=16
2107 https://bikroy.com/en/ad/walton-primo-good-phone-used-for-sale-dhaka
2108 https://bikroy.com/data/details?slug=xiaomi-redmi-note-5-black-used-for-sale-dhaka-235&similar_ads=16
2109 https://bikroy.com/data/details?slug=oppo-reno-6-official-8-128gb-used-for-sale-dhaka-1&similar_ads=16
2110 https://bikroy.com/data/details?slug=xiaomi-poco-m2-6-ram-64-rum-used-for-

<IPython.core.display.Javascript object>

In [81]:
text['ad']

{'id': '62263d6b0adbf2004b468f5b',
 'account': {'id': '59c8c371526a8e00016487da', 'traits': {}},
 'details': '',
 'slug': 'symphony-b6-used-for-sale-dhaka-595',
 'title': 'Symphony B6 (Used)',
 'description': 'All ok good condition',
 'properties': [{'label': 'Condition',
   'value': 'Used',
   'key': 'condition',
   'value_key': 'used'},
  {'label': 'Brand',
   'value': 'Symphony',
   'key': 'brand',
   'value_key': 'symphony'},
  {'label': 'Model',
   'value': 'B6',
   'key': 'model',
   'value_key': 'symphony-b6'},
  {'label': 'Features', 'value': 'Dual SIM', 'key': 'features'},
  {'label': 'Authenticity',
   'value': 'Original',
   'key': 'authenticity',
   'value_key': 'original'}],
 'isMember': False,
 'isAuthDealer': False,
 'isFeaturedMember': False,
 'type': 'for_sale',
 'status': 'deactivated',
 'membershipLevel': 'free',
 'isVerified': False,
 'postedDate': None,
 'isDoorstepDelivery': False,
 'isDSDApplicable': True,
 'deactivates': None,
 'location': {'id': 1292,
  'name':

In [86]:
jtext['adDetail']['data']['ad']

{'id': '62263d73a40329004a73ca8a',
 'account': {'id': '619a98a9cbac9b00486e08bd', 'traits': {}},
 'details': '',
 'slug': 'samsung-galaxy-s9-plus-2018-used-for-sale-dhaka-95',
 'title': 'Samsung Galaxy S9 Plus 2018 (Used)',
 'description': 'Real Box and Charger Available nZero Scratch, Full fresh phone',
 'properties': [{'label': 'Condition',
   'value': 'Used',
   'key': 'condition',
   'value_key': 'used'},
  {'label': 'Brand',
   'value': 'Samsung',
   'key': 'brand',
   'value_key': 'samsung'},
  {'label': 'Model',
   'value': 'Galaxy S9 Plus',
   'key': 'model',
   'value_key': 'samsung-galaxy-s9-plus'},
  {'label': 'Features',
   'value': 'Bluetooth, Camera, Dual-Lens Camera, Dual SIM, Expandable Memory, Fingerprint Sensor, GPS, Physical keyboard, Motion Sensors, 3G, 4G, GSM, Touch screen',
   'key': 'features'},
  {'label': 'Edition', 'value': '2018', 'key': 'edition'},
  {'label': 'Authenticity',
   'value': 'Original',
   'key': 'authenticity',
   'value_key': 'original'}],
 '

In [96]:
len(single_url)

2268

In [4]:
df = pd.DataFrame(all_data)
df

,ad_id,title,description,features,is_member,is_auth_dealer,is_featured_member,membership_level,is_verified,door_step_delivery,...,contact_name,phone_number,shop_name,shop_email,price,delivery_free,similar_ads,image_url,image_path,url
0,62249db047b4260019a7e76e,OnePlus 9 Pro 12GB 256GB global (New),@@ new and pack❤️😍nn14 days Replacement Guaran...,"{'Condition': 'New', 'Brand': 'OnePlus', 'Mode...",True,False,False,plus,False,False,...,Marvel,[01884002525],Marvel,marvelbd422@gmail.com,"Tk 63,990",False,"[{'id': '62249d02981cc0007af806c4', 'slug': 'o...",[https://i.bikroy-st.com/oneplus-9-pro-12gb-25...,[img/mobile/oneplus-9-pro-12gb-256gb-global-ne...,https://bikroy.com/en/ad/oneplus-9-pro-12gb-25...
1,621de793a951b2007b9519e4,OPPO A57 3/32 (New),Brand oppo\n\n\noppo a57\n\nOriginal \n\n=====...,"{'Condition': 'New', 'Brand': 'OPPO', 'Model':...",True,False,False,premium,True,False,...,Golden BD,"[01750663140, 01960323083, 01616323083]",Golden BD,mdrajibfakir972@gmail.com,"Tk 5,255",False,"[{'id': '622c43028e0c97007ccee0f5', 'slug': 'o...",[https://i.bikroy-st.com/oppo-a57-3-32-new-for...,[img/mobile/oppo-a57-3-32-new-for-sale-barisha...,https://bikroy.com/en/ad/oppo-a57-3-32-new-for...
2,622604016b9c5c001a0a814e,Samsung Galaxy J6 3/32 (Used),tas display nosto .pohne oky others no nay pmlem.,"{'Condition': 'Used', 'Brand': 'Samsung', 'Mod...",False,False,False,free,False,False,...,Аьія Ндззай Ядкіь,[01724069698],N/A,N/A,"Tk 2,000",False,"[{'id': '622c60a7cc3aaa00199980cf', 'slug': 's...",[https://i.bikroy-st.com/samsung-galaxy-j6-3-3...,[img/mobile/samsung-galaxy-j6-3-32-used-for-sa...,https://bikroy.com/en/ad/samsung-galaxy-j6-3-3...
3,622603f56b9c5c00490a816b,Realme C11 (Used),Realme C11 (Official).nRam: 2 GB. Rom: 32 GB.n...,"{'Condition': 'Used', 'Brand': 'Realme', 'Mode...",False,False,False,free,False,False,...,Prince Emu,[01866155789],N/A,N/A,"Tk 6,000",False,"[{'id': '621cc72f8854c24365c90a69', 'slug': 'r...",[https://i.bikroy-st.com/realme-c11-used-for-s...,[img/mobile/realme-c11-used-for-sale-sylhet-di...,https://bikroy.com/en/ad/realme-c11-used-for-s...
4,622603da14c5c30019b70d82,Samsung Galaxy A7 . (Used),Only Phone and Charger will be provided. Full ...,"{'Condition': 'Used', 'Brand': 'Samsung', 'Mod...",False,False,False,free,False,False,...,Nayem Ahmed Rafee,[01785274077],N/A,N/A,"Tk 9,000",False,"[{'id': '62175a5cd3f045007b9b5ed0', 'slug': 's...",[https://i.bikroy-st.com/samsung-galaxy-a7-201...,[img/mobile/samsung-galaxy-a7-2018-used-for-sa...,https://bikroy.com/en/ad/samsung-galaxy-a7-201...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,6225a26fc3d43300763db3c2,Walton Primo good phone (Used),1gb ram orgnalphone,"{'Condition': 'Used', 'Brand': 'Walton', 'Mode...",False,False,False,free,False,False,...,Shamim Reza,[01789793546],N/A,N/A,"Tk 5,000",False,"[{'id': '620547cc493bd000190a9a4f', 'slug': 'w...",[https://i.bikroy-st.com/walton-primo-good-pho...,[img/mobile/walton-primo-good-phone-used-for-s...,https://bikroy.com/en/ad/walton-primo-good-pho...
2107,622569ca253f3b004a2d1e80,Xiaomi Redmi Note 5 black (Used),phn a kuno prblam nay daikha neban.\n. (ram 3g...,"{'Condition': 'Used', 'Brand': 'Xiaomi', 'Mode...",False,False,False,free,False,False,...,Aksher Adrian,"[01830588974, 01877790774]",N/A,N/A,"Tk 6,500",False,"[{'id': '62022a0fa79c540019c14999', 'slug': 'x...",[https://i.bikroy-st.com/xiaomi-redmi-note-5-b...,[img/mobile/xiaomi-redmi-note-5-black-used-for...,https://bikroy.com/en/ad/xiaomi-redmi-note-5-b...
2108,6225a247eccded007787e86b,OPPO Reno 6 Official 8/128GB (Used),\n🔥🔥New Arrival - Best Price🔥🔥\n\n⚡BD Official...,"{'Condition': 'Used', 'Brand': 'OPPO', 'Model'...",True,False,False,premium,True,False,...,Gadget Wire,"[01933392919, 01715106793]",Gadget Wire,rayhan143m@gmail.com,"Tk 26,999",False,"[{'id': '622da6c1fbebf500191051e4', 'slug': 'o...",[https://i.bikroy-st.com/oppo-reno-6-official-...,[img/mobile/oppo-reno-6-official-8-128gb-used-...,https:/

In [90]:
df['image_path'][0]

['img/mobile/oneplus-9-pro-12gb-256gb-global-new-for-sale-chattogram@60e710cc-3497-417c-8f40-42630b9e4d97.jpg']

In [5]:
df.to_csv('csv/mobile.csv', index=False)